<a href="https://colab.research.google.com/github/mirsadra/casereport/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Required Packages  
The notebook is based on Google Colab.

In [1]:
!pip install biopython


In [2]:
from Bio import Entrez
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

# PubMed Data Fetching definition

In [3]:
def search(query):
    Entrez.email = 'miirsadra@gmail.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='publication date',
                            retmax='250000',
                            retmode='xml',
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'miirsadra@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

def fetch_data_for_query(query):
    studies = search(query)
    studiesIdList = studies['IdList']

    # Lists initialization
    author_list = []
    title_list = []
    abstract_list = []
    language_list = []
    pubdate_year_list = []
    keywords_list = []
    article_type_list = []

    chunk_size = 10000

    for chunk_i in range(0, len(studiesIdList), chunk_size):
        chunk = studiesIdList[chunk_i:chunk_i + chunk_size]
        papers = fetch_details(chunk)
        for paper in papers['PubmedArticle']:
        # Authors
            try:
                authors = paper['MedlineCitation']['Article']['AuthorList']
                if len(authors) > 1:
                    author_list.append(authors[0]['LastName'] + " et al.")
                else:
                    author_list.append(authors[0]['LastName'])
            except:
                author_list.append('No Author')

            # Title
            title_list.append(paper['MedlineCitation']['Article']['ArticleTitle'])

            # Abstract
            try:
                abstract_list.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
            except:
                abstract_list.append('No Abstract')

            # Language
            language_list.append(paper['MedlineCitation']['Article']['Language'][0])

            # Year
            try:
                pubdate_year_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])
            except:
                pubdate_year_list.append('No Data')

            # Keywords
            try:
                keywords = paper['MedlineCitation']['KeywordList']
                keywords_list.append(", ".join([kw for kw in keywords[0]]))
            except:
                keywords_list.append('No Keywords')

            # Article Type
            try:
                article_type_list.append(paper['MedlineCitation']['Article']['PublicationTypeList'][0])
            except:
                article_type_list.append('No Type')

        return pd.DataFrame(list(zip(author_list, title_list, abstract_list, language_list, pubdate_year_list, keywords_list, article_type_list)),
                        columns=['Authors', 'Title', 'Abstract', 'Language', 'Year', 'Keywords', 'Type'])

# PubMed Queries of our Case Report

In [4]:
# Query 1 for Table 1.
query1 = 'L-shaped renal ectopia OR L-shaped crossed renal ectopia OR L-shaped crossed fused AND Case Reports[PT]'
df1 = fetch_data_for_query(query1)

# Query 2 for Table 2.
query2 = 'single ureter AND crossed-fused AND Case Reports[PT]'
df2 = fetch_data_for_query(query2)

# Concatenate results into a single dataframe
df = pd.concat([df1, df2], ignore_index=True)

# Manual Adition of Information

### Tokgöz et al presented two L-shaped CFRE cases in one case report.
#### - Duplication of second case is done.

In [5]:
# Row 17 (Tokgöz et al) presented two cases.
# Duplication:
new_row = df.loc[17].copy()
df = pd.concat([df.iloc[:18], pd.DataFrame(new_row).T, df.iloc[18:]]).reset_index(drop=True)

### Adding Few Details Manually

In [6]:
case_details = [
    "Case Report: A 50-year-old male presented with hematuria, increased frequency, and burning sensation during micturition along with left flank pain for 2 months. There was a history of generalized weakness, weight loss, and loss of appetite during this period. On examination, the patient was poorly nourished with the presence of a palpable left-sided abdominal mass. Ultrasonography (USG) of the abdomen showed the presence of right ectopic kidney and left-sided hydronephrosis. A heterogeneous mass was noted in the left kidney. Another polypoidal mass was also seen in the lumen of the urinary bladder, attached to its posterolateral wall. Contrast-enhanced computerized tomography (CECT) of the abdomen helped in the renal anatomy and characterization of the mass, which revealed L-shaped crossed fused renal ectopia. The right kidney was not present in the right renal fossa and was in the midline, anterior to the aortic bifurcation at the L4-L5 level. It was malrotated and fused with the lower pole of the left kidney. The left kidney was enlarged, with a large soft heterogeneous tissue density mass involving the interpolar and lower pole regions that exhibited heterogeneous enhancement with central non-enhancing areas. Few calcified foci were seen in the mass, along with moderate hydronephrosis. In the delayed phase (15 minutes), no contrast excretion from the left kidney was recorded. The interpolar region of the right kidney was contiguously infiltrated by the left lower pole renal mass. Small tumor thrombi were present in the segmental right renal veins draining the interpolar region. Aortocaval, para-aortic and left renal hilar lymphadenopathy were also noted.",
    "Case Report: A 79-year-old woman with a history of surgery for peritonitis was referred to our hospital to treat an abdominal aortic aneurysm (AAA). Enhanced computed tomography (CT) revealed an AAA measuring 53 mm in diameter (Fig. 1A). The right kidney was located to the left of the midline; this condition is called L- shaped crossed fused renal ectopia (Fig. 1A and B). The patient",
    "Case Report: A 40-year-old woman, primigravida, first visited our hospital at 22 weeks due to a fever and left flank pain. These symptoms had begun 3 days prior to her visit. She reported several episodes of urinary-tract infections in the past. Her past surgical history was irrelevant to this case. She had no significant family history. Upon physical examination, the patient presented hyperthermia (39 °C), tachycardia (120 beats per minute), and blood pressure of 110/85 mmHg. There was suprapubic tenderness and left flank tenderness at percussion. Gynecological ex- amination was normal. Analytical evaluation revealed an elevated white blood cell (25*10) with 71% neutrophils, an elevated protein C reaction (150 mg/dl). Urine tested positive for leucocytes and protei- nuria but negative for red blood cells. The patient was admitted with the diagnostic of acute pyelonephritis and treatment was started em- pirically with ampicillin after urine and blood culture were performed. Despite treatment, the fever persisted and the patient's condition did not improve. Urine culture was positive for E. Coli. Antibiotics had been ineffective in controlling pyosepsis.",
    "X",
    "Case Report: A 79-year-old woman with systemic sarcoidosis donated her body to medical research at our department. She died from multiple organ failure resulting from hepatic cell carcinoma. Her liver was hypertrophied and indurated. During an anatomical dissection course for students, it was observed that she had L- shaped kidney. We were able to obtain prior clinical data, such as computed tomography images, from our hospital. Two images were available that had been tak- en on two previous occasions before she died, one at 2 months before her death and the other at 14 months.",
    "Cat",
    "Case Report: A 55-year-old woman presented with dull aching pain on the right side of her abdomen. Abdominal examination revealed an ill-defined lump in the right lumbar area extending to the hypogastrium and iliac fossa. Ultrasound revealed a right-sided renal mass with an empty left renal fossa with an ectopic kidney at the L4 level. Dedicated triphasic CECT of the abdomen was performed for preope- rative surgical planning. The CT revealed crossed fused renal ectopia on the right side. The right renal moiety was lying opposite the L1–L4 vertebrae. The left moiety was seen lying horizontally opposite the L4–L5 vertebrae with its pelvis lying anteriorly and forming an L-shaped crossed fused ectopia.",
    "Case Report: A 60-year-old female was admitted to our institution complaining of recurrent milky urine. The patient presented with an 8-month history of intermittent abdominal pain in the right flank region and chyluria, which had become worse in the past 4 months (presented as constant chyluria) with feebleness. Moderate pitting edema was observed in bilateral lower limbs. The urinary chyluria test was positive and 24-hour urine protein was 1.2 g. Night blood filarial examination and filarial serology were negative. The results of the routine blood test and liver and renal function were all within normal limits.",
    "Case Report: A 71-year-old man was evaluated at a local hospital due to a positive fecal occult blood test performed during a routine medical checkup. Detailed examination revealed sigmoid colon cancer, and the patient was referred to our hospital for treatment. He had a history of appendectomy for acute appendicitis 53 years ago and had been diagnosed with diabetes mellitus 3 years prior. Physical examination showed a soft and flat abdomen without palpable mass. Routine laborato- ry examination showed no abnormal values.",
    "Case Report: A 26-year-old male presented with a six-month history of intermittent abdominal pain in the left flank region. Medical workup, including urinalysis, was all within normal limits. A plain abdominal radiograph showed an enlarged left kidney shadow. The right kidney could not be visualized. An abdominal ultrasound examination demonstrated the absence of a right kidney and the presence of two kidneys in the left renal fossa, joined by parenchymal tissue.",
    "Case Report: A 16-year-old female was admitted with complaints of right-side lower abdominal pain of a dull aching type lasting 6 months. Physical examination revealed no palpable lumps. On further evaluation, ultrasonography revealed bilateral kidneys absent in the renal fossa and found in the pelvis. These findings were confirmed with an intravenous pyelogram, which showed the right kidney low lying in the lumbar and pelvic region, malrotated, and normally ex- creting and the left kidney pelvic in location, grossly hydronephrotic, and crossing the midline.",
    "Case Report: A 68-year-old woman with obesity, diabetes, hypertension, and ischemic heart disease presented with lower abdominal pain of 2 years’ duration. Computed tomography showed a left-to-right, L-shaped, crossed-fused kidney with staghorn calculi in both renal moieties. She was planned for supine percutaneous nephrolithotomy (PCNL) in view of her comorbidities and anatomic variation.",
    "Case Report: A 5-year-old girl was admitted into our hospital when she was found to have a palpable abdominal mass. She did not complain of abdominal pain or distension, fever, hematuria, nausea, vomiting or diarrhea. The physical examination revealed a palpable abdominal mass in her left abdomen. No family history of congenital diseases was reported. Laboratory findings were within normal limits.",
    "Case Report: A 68-year-old man had a pulsatile abdominal mass pointed out during a physical examination at an outpatient department after cerebral infarction. Ultrasound examination revealed an AAA, and he was referred to our hospital for surgery. Until then, he had not reported any sensation of pulsatile abdominal mass or complained of abdominal or back pain.",
    "Case Report: A 15-year-old girl presented with intermittent urinary incontinence as well as normal voiding. The physical examination revealed an intermittent drip of urine from the vaginal orifice. Laboratory tests showed no significant abnormalities. Filling the bladder with methylene blue via a urethral catheter, a clear fluid drained into the vulva. Ultrasonography and intravenous pyelography (IVP) indicated a large solitary malrotated left kidney. High-dose IVP, however, demonstrated a ureter originat- ing from the medial-inferior aspect of the “left kidney” crossing the midline at the L5-S1 level toward the pelvis.",
    "Case Report: A 58-year-old woman presented with fever and abdominal pain. Physical examination was notable for a febrile patient with tenderness over the abdomen and no flank knocking tenderness bilaterally. Laboratory tests showed normal renal function and numerous white and red blood cells in urine. A plain x-ray of the abdomen disclosed 1 radio-opaque lesion in an unusual medial position at the L3 vertebral level and another 2 radio-opaque lesions in the right pelvis. On computed tomography the radio-opacities turned out to be a right renal pelvic stone in a unique inverted L-shaped right-to-left cross-fused kidney",
    "Case Report: A 28-year-old man presented with a history of blunt abdominal and lower limb trauma in a road traffic accident sustained 3 days before. He had gross hematuria and a closed fracture of the shaft of the left femur. After initial resuscitation and stabilization, the patient underwent contrast-enhanced computed tomography (CECT) of the abdomen for evaluation of abdominal trauma. CECT revealed an empty right renal fossa. The right kidney was located in the pelvis and it was fused in L-shaped fashion to the left kidney. The left kidney was lying lower in position.",
    "Case 1: A 22-year-old male patient was admitted to our clinic with periumbilical pain referring to both flank area. On admission, serum biochemistry and complete blood count were normal. Urinalysis showed pyuria and hematuria and urine culture revealed E. coli (100,000 cfu/ml). After treatment of the infection with sensitive antibiotics excretory urography was obtained and a 2 · 1.4 cm opacity superimposed to the right transverse process of L4 vertebrae was seen at plain abdominal X-ray.",
    "Case 2: A 42-year-old female patient was referred with flank pain on the right. Her history revealed right pyelolithotomy 6 years ago and she was stone-free. On her last admission serum biochemical analysis, complete blood count and urine culture were totally normal. Urine sediment analysis showed microscopic hematuria. Excretory urography revealed an L-shaped kidney, multiple calculi in renal pelvis and calices of abnormally rotated right kidney.",
    "Case Report: A 71-year-old man presented with painless macroscopic hematuria. The physical examination was unremarkable, except for an enlarged prostate. The blood workup indicated normal renal function. An ultrasound study showed an ectopic, horizontally lying, kidney in the left lower abdomen and fused with the right kidney. The left renal pelvis, along with the proximal ureter, was significantly dilated. The cystoscopic examination was normal and revealed both ureteral orifices at their regular position. Computed tomography demonstrated an L-shaped crossed ectopic kidney with moderate left hydronephrosis",
    "X",
    "Case Report: A 67-year-old man presented himself at a local hospital with a 3-week history of appetite loss. He was suspected as having renal anomalies and a mass lesion on one kidney based on the findings of a CT scan. The patient was thereafter referred to us for further evalua- tion. On admission to National Beppu Hospital, Japan, the patient could describe no symptoms except for appetite loss, and he had no flank pain, fever or macrohematuria.",
    "X",
    "Case Report: A 32-year-old man consulted Osaka National hospital with chief complaints of dysuria and macrohematuria. DIP and CT revealed that the right kidney deviated to the lower pole of the left kidney and they fused together. The right ureter crossed over the supine. The calcified shadow existed in the lower end of the left ureter with cobra head image.",
    "Case Report: A 32-year-old man diabetic with macroscopic hematuria was referred to our department for clinical evaluation. The right kidney appeared to be medially dislocated at ultrasound examination, and both kidneys were posi- tioned lower than normal. Although the exact relationship of the kidneys to each other could not be established unequivocally due to the obese abdominal wall, those parts of the organs which could be visualized seemed to have normal sonographic structure.",
    "Case Report: A 38-year-old man complained of lower abdominal pain, frequency of micturition, and dysuria. Ureterectomy with vesiculeotomy on the right side was carried out. Histological finding is a transitional epithelium inside the duct. This case is the 11th reported case in Japan and some statistical studies were carried out. ",
    "X",
    "A significantly dilated extra-renal pelvis is seen draining both collecting systems and emptying into a single ureter., The arterial supply is a single artery from the abdominal aorta at its bifurcation. The venous drainage of the right and upper moiety is to the confluence of the common iliac veins. The left and lower moiety drains to the inferiorvena cava posterior to aorta",
    "Crossed right-to-left inferior renal ectopia with both kidneys placed on the left side of the body associated with proximal ureteral calculus.",
    "Second kidney on the right side fused to the lower pole of the right kidney. the left ureter was absent. The fused kidneys were supplied by two renal arteries originating from the left internal iliac artery.",
    "Complain of Intermittent pain abdomen, empty left renal fossa and left kidney was present on the right side, below the right kidney. Left renal fossa was empty with abnormal rotation of left kidney and fusion to right kidney at midpole level likely suggestive of Crossed fused ectopia. Single ureter was noted on right side with normal excretion of contrast,",
    "A single kidney with 2 ureters, and the orifices of the ureters were connected at both sides of bladder., CT were performed and revealed crossed fused renal ectopia.",
    "X",
    "Presented with continuous urinary incontinence since birth, CT revealed a disk-shaped hypodense mass, A fused-crossed ectopic right kidney with vaginal ectopic ureter was impressed.",
    "Caudal regression syndrome associated with crossed fused renal ectopia, fusion of the mid-third of the ureters and abdominal aortic anomalies, CT revealed crossed renal ectopia with fusion and severe hydronephrosis, Two distinct ureters with only midway segmental fusion have not been previously reported.",
    "Single system ectopic ureter draining into the mesonephric duct cyst and ipsilateral crossed fused kidney",
    "X",
    "X",
    "A single renal mass was seen on the right side with a double collecting system draining the mass. This duplex system opened into a large spherical mass just inferior to the kidney. A single ureter was visualized below this mass",
]

df['Case Reports'] = case_details

# Drop unrelated rows

In [7]:
# Drop X and Cat rows under Case Reports:
df = df[~df['Case Reports'].isin(["X", "Cat"])]

In [8]:
df

,Authors,Title,Abstract,Language,Year,Keywords,Type,Case Reports
0,Kumar et al.,Primary Chondrosarcoma in L-shaped Crossed Fus...,Primary renal chondrosarcomas are rare tumors ...,eng,2022,"chondrosarcoma, kidney, renal ectopia, urinary...",Case Reports,Case Report: A 50-year-old male presented with...
1,Kawatani et al.,Endovascular aortic aneurysm repair with rever...,Crossed fused renal ectopia is a very rare con...,eng,2021,No Keywords,Case Reports,Case Report: A 79-year-old woman with a histor...
2,Jallouli et al.,About a case of ureteropelvic junction obstruc...,No Abstract,eng,2018,"Ectopia, Kidney, Pyloplasty",Case Reports,"Case Report: A 40-year-old woman, primigravida..."
4,Iwanaga et al.,Anatomical and Radiological Analyses of L-shap...,L-shaped kidney is a congenital anomaly. The d...,eng,2018,"L-shaped kidney, crossed fused renal ectopia, ...",Case Reports,Case Report: A 79-year-old woman with systemic...
6,Kumar et al.,Robot-assisted heminephrectomy for chromophobe...,Renal cell carcinoma associated with fused ect...,eng,2015,"Nephrectomy, Renal cell carcinoma, Robotics",Case Reports,Case Report: A 55-year-old woman presented wit...
7,Liu et al.,"Right-Crossed, Fused Renal Ectopia L-Shaped Ki...",Crossed fused renal ectopia combined with chyl...,eng,2015,No Keywords,Case Reports,Case Report: A 60-year-old female was admitted...
8,Nakai et al.,Laparoscopic sigmoid colectomy for a patient w...,Crossed-fused renal ectopia (CFRE) is a very r...,eng,2015,"Colon cancer, Crossed renal ectopia, Laparosco...",Case Reports,Case Report: A 71-year-old man was evaluated a...
9,Pupca et al.,Left crossed fused renal ectopia L-shaped kidn...,Crossed fused renal ectopia (CFRE) is the seco...,eng,2014,No Keywords,Case Reports,Case Report: A 26-year-old male presented with...
10,Muruganandham et al.,Laparoscopic pyeloplasty for ureteropelvic jun...,Crossed fused renal ectopia is a rare anomaly ...,eng,2014,"Laparoscopy, Plastic surgery, Urogenital abnor...",Case Reports,Case Report: A 16-year-old female was admitted...
11,Mishra et al.,Supine percutaneous nephrolithotomy for bilate...,A 68-year-old woman was diagnosed to have a le...,eng,2013,No Keywords,Case Reports,"Case Report: A 68-year-old woman with obesity,..."


# Creating a New Table

In [9]:
data = {
    "No.": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],

    "Author(s)": ["M Kumar et al.", "Y Kawatani et al.", "W Jallouli et al.",
                  "J Iwanaga et al.", "S Kumar et al.", "D Liu et al.",
                  "N Nakai et al.", "G Pupca et al.", "K Muruganandham et al.",
                  "S Mishra et al.", "W Song et al.", "Y Iida et al.", "L Liu et al.",
                  "SD Chung et al.", "S Kumar et al.", "H Tokgöz et al.", "H Tokgöz et al.",
                  "U Gur et al.", "H Tsunoe et al.", "Y Tsujimoto et al.",
                  "E Szentgyörgyi et al.", "K Arai et al.", "K Habtemichael et al.",
                  "J Alić et al.", "S Khanduri et al.", "P Shaha et al.",
                  "EH Moon et al.", "VC Lin et al.", "YC Duh et al.",
                  "S Kamata  et al.", "S Khanduri et al."],

    "Year": [2022, 2021, 2018, 2018, 2015, 2015, 2015, 2014, 2014, 2013,
             2012, 2010, 2010, 2009, 2008, 2005, 2005, 2003, 2001, 1999, 1995,
             1989, 2021, 2020, 2019, 2016, 2015, 2010, 2007, 2000, 1975],

    "Age/Gender": ["50/M", "79/F", "40/F", "79/F", "55/F", "60/F", "71/M",
                   "26/M", "16/F", "68/F", "5/F", "68/M", "15/F", "58/F", "28/M",
                   "22/M", "42/F", "71/M", "67/M", "32/M", "32/M", "38/M",
                   "17/M", "53/M", "22/M", "22/F", "20/M", "4.5/F", "NB",
                   "5/F", "3/M"],

    "Type of Crossed-Fused Kidney": ["L", "L", "L", "L", "L", "L", "L", "L",
                                     "L", "L", "L", "L", "L", "L", "L", "L", "L",
                                     "D", "L", "Inverted L", "L", "L", "Pancake",
                                     "Inferior", "Inferior", "S-shaped*",
                                     "Unexplained", "Disc-shape", "Unexplained",
                                     "Single", "-"],

    "Chief Complaint": ["Hematuria, LFP", "AAA", "LFP, Fever", "Dead",
                              "Abdominal Pain", "Milky urine", "Routine Checkup",
                              "LFP", "Abdominal Pain", "Abdominal Pain",
                              "Palpable Abdominal Mass", "AAA", "UI", "Abdominal Pain",
                              "Hematuria", "Flank Pain", "Flank Pain", "Macroscopic Hematuria",
                              "Loss of Appetite", "Dysuria, Macrohematuria", "Macroscopic Hematuria",
                              "Dysuria, Abdominal Pain", "Abdominal Pain, Urinary Retention",
                              "Abdominal Pain", "Abdominal Pain, Burning Micturition",
                              "Abdominal Pain", "Incidentally", "Urinary Incontinence",
                              "-", "-", "Hematuria"],

    "Imaging Used": ["USG, CECT", "CECT", "CT", "Dissection", "CECT",
                     "CT, CT Angiography", "Sigmoid Colon Cancer",
                     "MDCT, Abdominal X-Ray", "CECT, SPECT, CTA", "CT",
                     "Ultrasonography, CT", "MDCT, Ultrasound",
                     "Ultrasonography, IVP, 3D-CT", "CT, Abdominal X-Ray", "CECT",
                     "Abdominal X-Ray, Ultrasonography", "Ultrasonography",
                     "CT, Ultrasound", "CT", "CT", "CT", "-", "CTU, Ultrasonography",
                     "CECT, 3D-CT", "MDCT", "CECT, CTA", "MDCT, SPECT",
                     "CT", "CT", "3D-CT", "IVP" ],

    "No. of Ureters": ["Normal", "Normal", "Normal", "Normal", "Normal", "Normal",
                       "Single", "Normal", "Normal", "Normal", "Normal",
                       "Normal", "Double-Y inverted", "Normal", "Normal", "Normal",
                       "Normal", "Normal", "Normal",
                       "Normal", "Normal", "-", "Single", "Normal", "Single", "Single",
                       "Normal", "SSEU (Vaginal)", "Mix*", "Single", "Single"],

    "PMID": [34976575, 34168859, 30105208, 28603157, 26495075,
                         26066307, 25785321, 25607413, 25405020, 23273097,
                         22056611, 21035712, 20347478, 18692871, 18429685,
                         16362579, 16362579, 14550458, 11555015, 10658464,
                         8775033, 2678982, 34354929, 33194464, 30931185,
                         27134964, 26252333, 19931894, 17968797, 10806577,
                         1159928]
}

# LFP: Left Flank Pain
# AAA: Abdominal Aortic Aneurysm
# RCC: Renal Cell Carcinoma
# FOBT: Fecal occult blood test
# NCS: The Nutcracker Syndrome
# CTA: CT Angiography
# UPJ: Ureteropelvic Junction
# UI: Urinary Incontinence
# IVP: Intravenous Pyelography
# TCC: Transitional Cell Cancer
# * The auhors avoid to classify due to presence of single ureter.
# SSEU: A single-system ectopic ureter
# NB: New Born


In [10]:
df_data = pd.DataFrame(data)

### Further data clearance

In [12]:
df_data = df_data[df_data['PMID'] != 28603157]  # Dissection report
df_data = df_data[df_data['PMID'] != 33194464]  # Was not `single` ureter.
df_data = df_data[df_data['PMID'] != 26252333]  # Duplex kidney. Was not CFRE.
df_data = df_data[df_data['PMID'] != 10806577]  # Japanese case. Lack of DOI and Manuscript.
df_data = df_data[df_data['PMID'] != 1159928]   # Despite the presence of CFRE and single ureter, The report did not specify the type and description resembles Duplex kidney.

### Adding `Türkvatan A, Olçer T, Cumhur T. Multidetector CT urography of renal fusion anomalies. Diagn Interv Radiol. 2009 Jun;15(2):127-34.`.
#### PMID: 19517383.  
#### Case No. 4 is a 23-year-old male with L-shaped CFRE.
16, "A Türkvatan et al", 2008, "23/M", "L", "Lumbar pain", "Imaging Used": "MDCT", "Normal", 19517383

In [14]:
df_top = df_data.iloc[:15]
df_bottom = df_data.iloc[15:]

new_data = {
    "No.": [16],
    "Author(s)": ["A Türkvatan et al"],
    "Year": [2008],
    "Age/Gender": ["23/M"],
    "Type of Crossed-Fused Kidney": ["L"],
    "Chief Complaint": ["Lumbar pain"],
    "Imaging Used": ["MDCT"],
    "No. of Ureters": ["Normal"],
    "PMID": [19517383]
}

df_new_row = pd.DataFrame(new_data)
df_data = pd.concat([df_top, df_new_row, df_bottom], ignore_index=True)